In [ ]:
import argparse
import io
from google.oauth2 import service_account
from google.cloud import vision
import numpy as np
import imageio
from base64 import b64encode
import json
import requests
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
vision_client = vision.ImageAnnotatorClient()
path = 'datasets/KAIST/English/Digital_Camera/DSC02629.JPG'

plt.imshow(cv2.imread(path))

In [ ]:
f = open("datasets/KAIST/English/Digital_Camera/DSC02629.xml", "r")
print(f.read())

In [ ]:
with io.open(path, 'rb') as image_file:
    content = (image_file.read())
image = vision.types.Image(content=content)

In [1]:
text_detection_response = vision_client.text_document_detection(image=image)

NameError: name 'vision_client' is not defined

In [ ]:
text_detection_response

In [ ]:
text_detection_response.text_annotations[0].description

In [ ]:
text_detection_response.full_text_annotation.text

In [ ]:
 #, image_context={"language_hints": ["ko"]}

text_detection_response.text_annotations

In [ ]:
f = open("datasets/MSRA-TD500/test/IMG_1557.gt", "r")
print(f.read())

In [ ]:
from xml.dom import minidom
xmldoc = minidom.parse('datasets/KAIST/English/Digital_Camera/DSC02629.xml')
itemlist = xmldoc.getElementsByTagName('word')

In [ ]:
for word in itemlist:
    print (word.value)

In [ ]:
itemlist

In [ ]:
%%bash
sudo pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
import xml.etree.ElementTree as ET 
tree = ET.parse('datasets/KAIST/English/Digital_Camera/DSC02629.xml') 
root = tree.getroot() 

In [ ]:
ground_truth = ''
for elem in tree.iter(tag='character'):
    ground_truth += (elem.attrib['char'])

In [ ]:
ground_truth

In [13]:
def simple_tokenize(source_str, token_delim=' ', seq_delim='\n'):
    return filter(None, re.split(token_delim + '|' + seq_delim, source_str))

from mxnet import nd
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

def norm_vecs_by_row(x):
    return x / nd.sqrt(nd.sum(x * x, axis=1) + 1E-10).reshape((-1,1))

def get_knn(vocab, k, word):
    word_vec = vocab.embedding[word].reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_vec)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k+1, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    # Remove unknown and input tokens.
    return vocab.to_tokens(indices[1:])

In [16]:
import gluonnlp as nlp
import re
text = 'ICE COFFEE'
glove_6b50d = nlp.embedding.create('glove', source='glove.6B.50d')
vocab = nlp.Vocab(nlp.data.Counter(glove_6b50d.idx_to_token))
vocab.set_embedding(glove_6b50d)

In [19]:
get_knn(vocab, 5, 'ICE COFFEE')

['!', '<pad>', '<unk>', '<bos>', '!!']

In [30]:
from nltk.translate.bleu_score import sentence_bleu
score = sentence_bleu(['ICE', 'COFFEE'], ['ICE', 'COFFEE'])
score

0

In [11]:
counter

Counter({'ICE': 1, 'COFFEE': 1})

In [109]:
import numpy as np
def cosine_distance(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')


In [111]:
cosine_distance('ICY COFFEE', 'ICE COFFEE')

NameError: name 'preprocess' is not defined

In [92]:
import numpy as np
def jaccard_similarity(translated, original):
    intersection = len(list(set(translated).intersection(set(original))))
    union = min(len(translated), len(original))
    return (intersection / union)

In [88]:
len(translated), len(original)

(3, 3)

In [93]:
jaccard_similarity('ICE', 'ICY')

1.0

In [100]:
np.ceil(6.7)

7.0

In [107]:
custom_ceil(.49)

0.5

In [72]:
def custom_ceil(float_num):
    return np.ceil(float_num*10)/10

translated, original = 'ICE', 'ICY'
intersection = set(query).intersection(set(document))
union = set(query).union(set(document))
intersection, union

({'C', 'I'}, {'C', 'E', 'I', 'Y'})

In [78]:
intersection = len(list(set(translated).intersection(set(original))))
intersection

2

In [76]:
len(type(intersection))

TypeError: object of type 'type' has no len()

In [41]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

In [56]:
sentence1, sentence2 = 'ICY COFFEE.', 'ICED COFFET'
sentence1 = pos_tag(word_tokenize(sentence1))
sentence2 = pos_tag(word_tokenize(sentence2))
synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 

In [57]:
synsets2

[None, None]

In [ ]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    print (score, count)
    #score /= count
    return score
 
sentences = [
    "Dogs are awesome.",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "Cats are beautiful animals.",
] 

In [40]:
focus_sentence = "Cats are beautiful animals."
for st in sentences: 
    print (sentence_similarity(focus_sentence, st))

0.0 0
0.0
0.0 0
0.0
0.0 0
0.0
0.0 0
0.0


In [35]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [51]:
from nltk.corpus import wordnet as wn
 
print (wn.synsets('cat', 'n'))
# [Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01')]
 
print (wn.synsets('dog', 'n'))
# [Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01')]
 
print (wn.synsets('feline', 'n'))
# [Synset('feline.n.01')]
 
print (wn.synsets('mammal', 'n'))
# [Synset('mammal.n.01')]
# It's important to note that the `Synsets` from such a query are ordered by how frequent that sense appears in the corpus
 
# You can check out how frequent a lemma is by doing:
cat = wn.synsets('cat', 'n')[0]     # Get the most common synset
#print cat.lemmas()[0].count()       # Get the first lemma => 18
 
 
 
dog = wn.synsets('dog', 'n')[0]           # Get the most common synset
feline = wn.synsets('feline', 'n')[0]     # Get the most common synset
mammal = wn.synsets('mammal', 'n')[0]     # Get the most common synset
 
 
 
# You can read more about the different types of wordnet similarity measures here: http://www.nltk.org/howto/wordnet.html
for synset in [dog, feline, mammal]:
    print ("Similarity(%s, %s) = %s" % (cat, synset, cat.wup_similarity(synset)))
 
# Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.2
# Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.5
# Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.2

[Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01')]
[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01')]
[Synset('feline.n.01')]
[Synset('mammal.n.01')]
Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.8571428571428571
Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.9629629629629629
Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.8333333333333334


In [50]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True